In [7]:
import numpy as np
import cv2

import os
import random
import pickle
import time

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import regularizers
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam


In [3]:
pickle_X_in = open("X.pickle", "rb")
X = pickle.load(pickle_X_in)

pickle_y_in = open("y.pickle", "rb")
y = pickle.load(pickle_y_in)

X = X/255.0

X_train, X_val,y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=241)
y_train = np.array(y_train)
y_val = np.array(y_val)

# Find the best layer size

In [8]:
layer_sizes = [64, 128, 256]

for layer_size in layer_sizes:

   name = "{}-layer_size-{}".format(layer_size, int(time.time()))
   model = Sequential([
                Conv2D(layer_size, (2, 2), activation="tanh", kernel_regularizer=regularizers.l2(0.01)),
                AveragePooling2D(pool_size=(2, 2)),

                Conv2D(layer_size, (2, 2), activation="tanh", kernel_regularizer=regularizers.l2(0.01)),
                AveragePooling2D(pool_size=(2, 2)),

                Conv2D(layer_size, (2, 2), activation="tanh", kernel_regularizer=regularizers.l2(0.01)),
                AveragePooling2D(pool_size=(2, 2)),

                Flatten(),
                Dense(layer_size, activation="tanh"),
                Dense(1, activation="sigmoid")
   ])

   learning_rate_schedule = ExponentialDecay( 0.0001, 1500, 0.8 )

   opt = Adam(learning_rate=learning_rate_schedule)

   model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics = ['accuracy']
   )
   tensorboard = tf.keras.callbacks.TensorBoard(log_dir="layer_size_logs/{}".format(name))
   history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val), shuffle=True,callbacks=[tensorboard])




Epoch 1/5
6/6 [==============================] - 8s 771ms/step - loss: 1.9753 - accuracy: 0.5341 - val_loss: 1.9323 - val_accuracy: 0.6000
Epoch 2/5
6/6 [==============================] - 4s 687ms/step - loss: 1.8999 - accuracy: 0.7273 - val_loss: 1.8666 - val_accuracy: 0.6500
Epoch 3/5
6/6 [==============================] - 4s 713ms/step - loss: 1.8521 - accuracy: 0.7614 - val_loss: 1.8364 - val_accuracy: 0.6000
Epoch 4/5
6/6 [==============================] - 4s 686ms/step - loss: 1.8075 - accuracy: 0.7216 - val_loss: 1.7478 - val_accuracy: 0.7500
Epoch 5/5
6/6 [==============================] - 4s 677ms/step - loss: 1.7936 - accuracy: 0.6989 - val_loss: 1.7405 - val_accuracy: 0.7500
Epoch 1/5
6/6 [==============================] - 13s 2s/step - loss: 3.2141 - accuracy: 0.6250 - val_loss: 3.2337 - val_accuracy: 0.6000
Epoch 2/5
6/6 [==============================] - 10s 2s/step - loss: 3.1237 - accuracy: 0.6534 - val_loss: 3.0176 - val_accuracy: 0.8000
Epoch 3/5
6/6 [================

## Conclusion

In [3]:
'''
After examining the graphs provided by Tensorboard, we find that the model performs best with
128 neurons in each layer
'''